In [1]:
import os
import sys
import random
import time
from random import seed, randint
import argparse
import platform
from datetime import datetime
import imp
import numpy as np
import fileinput
from itertools import product
import pandas as pd
from scipy.interpolate import griddata
from scipy.interpolate import interp2d
import seaborn as sns
from os import listdir

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import griddata
import matplotlib as mpl
# sys.path.insert(0,'..')
# from notebookFunctions import *
# from .. import notebookFunctions
from Bio.PDB.Polypeptide import one_to_three
from Bio.PDB.Polypeptide import three_to_one
from Bio.PDB.PDBParser import PDBParser
from pyCodeLib import *
# from small_script.myFunctions import *
sys.path.insert(0, "/Users/weilu/openmmawsem")
from helperFunctions.myFunctions import *
from collections import defaultdict
%matplotlib inline
# plt.rcParams['figure.figsize'] = (10,6.180)    #golden ratio
# %matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
plt.rcParams['figure.figsize'] = np.array([16.18033, 10])    #golden ratio
plt.rcParams['figure.facecolor'] = 'w'
plt.rcParams['figure.dpi'] = 100
plt.rcParams.update({'font.size': 22})

In [42]:
structure = parse_pdb("/Users/weilu/Research/server/nov_2019/cys_1fs3/setups/1fs3/1fs3-cleaned")

In [43]:
def calculate_cys_density(res_list, neighbor_list, min_seq_sep=2, k_bin=5):
    num_residues = len(res_list)
    density = np.zeros(num_residues)
    for res1globalindex, res1 in enumerate(res_list):
        res1index = get_local_index(res1)
        res1chain = get_chain(res1)
        if res1.get_resname() != "CYS":
            continue
        for res2 in get_neighbors_within_radius(neighbor_list, res1, 9.0):
            if res2.get_resname() != "CYS":
                continue
            res2index = get_local_index(res2)
            res2chain = get_chain(res2)
            res2globalindex = get_global_index(res_list, res2)
            if abs(res2index - res1index) >= min_seq_sep or (res1chain != res2chain):
                rij = get_interaction_distance(res1, res2)
                density[res1globalindex] += 0.5*(1-np.tanh(k_bin*(rij-4.2)))
    return density

In [75]:
k_bin = 5
step_k_bin = 20
min_seq_sep = 2
e_cys = 0
res_list = get_res_list(structure)
neighbor_list = get_neighbor_list(structure)
cys_density = calculate_cys_density(res_list, neighbor_list, min_seq_sep=2, k_bin=k_bin)


for res1globalindex, res1 in enumerate(res_list):
    res1index = get_local_index(res1)
    res1chain = get_chain(res1)
    rho_i = cys_density[res1globalindex]
    if res1.get_resname() != "CYS":
        continue
    for res2 in get_neighbors_within_radius(neighbor_list, res1, 9.0):
        if res2.get_resname() != "CYS":
            continue
        res2index = get_local_index(res2)
        res2chain = get_chain(res2)
        res2globalindex = get_global_index(res_list, res2)
#         if True:
#         if res2globalindex - res1globalindex:
#         if res2globalindex - res1globalindex != 0:
        if res2globalindex - res1globalindex >= min_seq_sep or (res1chain != res2chain and res2globalindex > res1globalindex):
            rho_j = cys_density[res2globalindex]
            rij = get_interaction_distance(res1, res2)
            theta = -0.5*(1-np.tanh(k_bin*(rij-4.2)))
            stepNear=0.5*(np.tanh(step_k_bin*(0.5-abs(rho_i-rho_j)))+1)
            stepSmall=0.5*(np.tanh(step_k_bin*(2.5-rho_i-rho_j))+1)
            print(res1globalindex, res2globalindex, stepNear, stepSmall, theta)
            e_cys += stepNear * stepSmall * theta
e_cys


25 83 0.9999999979388463 0.9999999984124728 -0.9967363965970196
25 94 0.9999999966725845 0.999999999016613 -0.0
39 83 0.9999999966725845 0.999999999016613 -0.0
39 94 0.9999999979388463 0.999999999390845 -0.9847631452932324
57 71 0.9999268220779248 0.9999999999999756 -0.0
57 109 0.9999999979388463 0.9999999991340194 -0.989160517696404
64 71 0.9999999979388463 1.0 -0.7272241080185597
64 109 0.9999268220779248 0.9999999999999756 -0.0
71 109 0.9999268220779248 0.9999999999999756 -1.942890293094024e-15
83 94 0.9999999966725845 0.999999999016613 -0.0


-3.6978841569444967

In [76]:
len(np.arange(-40, 40, 4))

20

In [77]:
len(np.arange(0, 180, 15))

12

In [45]:
cys_density

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.9967364 , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.98476315,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.98916052, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.72722411,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.72722411, 0.        , 0.        , 0.     

1.8841564833720896

In [ ]:
step(abs(cysResId1-cysResId2)-2)

In [14]:
cys_density

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.99998928, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.99976066,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.99987993, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.87665933,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.87665933, 0.        , 0.        , 0.     

In [6]:
all_res = list(structure.get_residues())

In [7]:
res = all_res[0]

In [8]:
res.get_resname() == "CYS"

'LYS'